# Optimizing Ski Students Pickup

A ski school provides transportation to the ski resort for its students.

The ski school has $k$ buses, each one having the capacity to transport at most $C$ students.
Based on enrolments for the next winter, the school expects to pick up its students from $n$ neighbouring towns.
Each town has $d_i$ students, and it must be visited exactly once by one of the $k$ buses.

A bus visiting a town must pick up all of its students, and the total number of students picked up by each bus must not exceed its capacity.
Lastly, it is mandatory that each bus starts and ends its route at the ski school.

**Your goal is to develop a model to plan the pickup routes of the ski school with the goal of minimizing transportation costs.**


## Formulation

### Sets

- $V$: set of $n$ neighbouring towns to be visited.
- $K$: set of $k$ available buses.

### Parameters

- $c_{ij}$: distance between towns $i \in V$ and $j \in V$,
- $d_i$: number of students waiting at town $i \in V$,
- $C$: maximum number of students per bus.

### Variables

- $x_{ijk}$: binary variable whose value is $1$ if bus $k \in K$ travels from town $i \in V$ to $j \in V$, 0 otherwise,
- $y_{ik}$: binary variable whose value is 1 if bus $k \in K$ visits town $i \in V$.

### Model

$$
\begin{array}{lll}
   \min & \sum_{i \in V} \sum_{j \in V} c_{ij} \sum_{k \in K} x_{ijk}\\
   \textrm{s.t.} & \sum_{k \in K} y_{ik} = 1 & \forall i \in V 	\setminus \{0\}\\
                 & \sum_{k \in K} y_{0k} = K & \\
                 & \sum_{j \in V} x_{ijk} = \sum_{j \in V} x_{jik} & \forall i \in V, k \in K \\
                 & \sum_{j \in V} x_{ijk} = y_{ik} & \forall i \in V, k \in K \\
                 & \sum_{i \in V} d_i y_{ik} \le C & \forall k \in K \\
                 & \sum_{i \in S} \sum_{j \in S} x_{ijk} \le |S| - 1 & \forall S \subseteq V 	\setminus \{0\}, |S| \ge 1, k \in K \\
                 & y_{ik} \in \{0,1\} & \forall i \in V, k \in K \\
                 & x_{ijk} \in \{0,1\} & \forall i \in V, j \in V, k \in K
\end{array}
$$


In [1]:
%pip install mip

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import math
import networkx as nx
from itertools import chain, combinations
import mip


n = 6   # number of towns
k = 2   # number of buses
C = 20  # max number of students per bus

# number of students waiting in each town
d = [2, 3, 7, 8, 8, 8]

V = range(n)  # set of towns. The first town (index 0) contains the ski resort.
V0 = V[1:]    # set of towns without the ski resort.
K = range(k)  # set of buses.
towns = [1,2,3,4,5,6]


np.random.seed(12345)
grid_size = 100
point = grid_size * np.random.random((n,2))

# distance between each pair of towns
c = np.array([[math.sqrt(np.sum((point[i] - point[j])**2)) for i in V] for j in V])

In [3]:
import mip
m = mip.Model()

# Add variable x_ijk whose value is 1 if bus k travels from town i to j 
x = {}
for i in range(n):
  for j in range(n):
    for k_i in range(k):
      x[(i, j, k_i)] = m.add_var(var_type=mip.BINARY)

# Add variable y_jk whose value is 1 if bus k visits town i
y = {}
for i in range(n):
  for k_i in range(k):
      y[(i, k_i)] = m.add_var(var_type=mip.BINARY)

# Setup objective function to minimize
m.objective = mip.minimize(mip.xsum(c[i, j] * x[(i, j, k_i)] for i in range(n) for j in range(n) for k_i in range(k)))

# Constrain that each town has to be reached by only one of the busses
for i in range(1, n):  
  m.add_constr(mip.xsum(y[(i, k_i)] for k_i in range(k)) == 1)

# Constrain that each buss has to start at the ski resort
m.add_constr(mip.xsum(y[(0, k_i)] for k_i in range(k)) == k)

# There should be the same number of busses going in one direction as in the other direction
for i in range(n):
  for k_i in range(k):
    m.add_constr(mip.xsum(x[(i, j, k_i)] for j in range(n)) == mip.xsum(x[(j, i, k_i)] for j in range(n)))

# A buss should travel to the city it visits
for i in range(n):
  for k_i in range(k):
    m.add_constr(mip.xsum(x[(i, j, k_i)] for j in range(n)) == y[(i, k_i)])

# A buss should not have more people aboard than its max capacity
for k_i in range(k):
  m.add_constr(mip.xsum(d[i]*y[(i, k_i)] for i in range(n)) <= C)

# Check that there are no cycles, remove {0}
max_subset = set(range(1, n))
for S in chain.from_iterable(combinations(max_subset, r) for r in range(len(max_subset)+1)):
  if len(S) >= 1:
    for k_i in range(k):
      m.add_constr(mip.xsum(x[(i, j, k_i)] for i in S for j in S) <= len(S)-1)

m.optimize()

buss_cities = {}
for i in range(n):
  for k_i in range(k):
    if y[(i, k_i)].x:
      if k_i not in buss_cities:
        buss_cities[k_i] = []
      buss_cities[k_i].append(i)
print(buss_cities)

buss_routes = {}
for k_i in range(k):
  buss_routes[k_i] = []
  finished = False
  i = 0
  while(not finished):
    for j in range(n):
      if x[(i, j, k_i)].x:
        buss_routes[k_i].append(f'{i} => {j}')
        i = j
        if j == 0:
          finished = True
        break
print(buss_routes)


{0: [0, 3, 5], 1: [0, 1, 2, 4]}
{0: ['0 => 3', '3 => 5', '5 => 0'], 1: ['0 => 1', '1 => 2', '2 => 4', '4 => 0']}
